In [2]:
import pandas as pd
import datetime as dt
import numpy as np
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector
import os


def run_brand_search_query(treatment_metro, treatment_region):
    # Establish connection to Snowflake
    conn = snowflake.connector.connect(account ='justworks-main',
                                            user= 'lmola@justworks.com',
                                            authenticator="externalbrowser")
    cur = conn.cursor()

    cur.execute("USE DATABASE prod_ent_db;")
    
    # Define the query
    brand_query = """ 
    with campaign_spend as (select state, region, metro, sum(m.cost_usd) as spend,
    sum(m.impression_count) as impressions,sum(m.click_count) as clicks,
    DATE_TRUNC('MONTH', to_date(m.metric_date)) AS month_start


    from  prod_analytics_db.marketing.fact_marketing_performance_daily_ad_metrics m 

    join prod_analytics_db.marketing.dim_marketing_performance_campaigns c

    on m.campaign_id = c.campaign_id

    join prod_analytics_db.marketing.dim_marketing_performance_ad_groups ad
    on m.ad_group_id = ad.ad_group_id

    left join (Select fiscal_year_quarter
    , r.month_name
    , r.date,day_of_week_name
    ,fiscal_week_num
    ,day_of_fiscal_quarter_num
    ,fiscal_quarter
    ,fiscal_year
    ,first_value(r.date) over(partition by iso_running_week order by r.date asc) AS week_of_qtr

    from community_db.ent_static.reference_dates r)ro

    on ro.date= m.metric_date


    where metric_date >= '2022-06-01'

    and ad.type in ('SEARCH_STANDARD')

    and (campaign_name like '%Brand%'
    or campaign_name like '%BRAND%'
    or campaign_name like '%BRD%' 
    or campaign_name like '%branded%'
    or campaign_name like '%justworks%'
    or campaign_name like '%just%')
    

    group by state, region, metro, month_start)
    --spend_month, spend_year)
    --7, 8, 9, 10, 11, 12, 13, 14, 15, 16)
    --4,5,6,7,8,9,10,11,12,13)

    select * from campaign_spend ;

    """
    
    # Execute the query and load results into a DataFrame
    df = pd.read_sql(brand_query, conn)

    # Close the connection
    conn.close()
    
    final_df = df[(df['METRO'] == treatment_metro) & (df['REGION'] == treatment_region)]

    return final_df


In [3]:
brand_search_df = run_brand_search_query('New York, NY', 'New York')
#direct_search_df = run_direct_search_query('New York', 'New York')
brand_search_df.head()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://justworks.cloudflareaccess.com/cdn-cgi/access/sso/saml/d74bc06bca5abf66677a16a11e6235e70f0ac75f0e462ccdea97b5bcb5cab993?SAMLRequest=lZNfb9owFMW%2FSuQ9J3YCSYgFVKyMLVW7okIrxpvjOODh2KntkPLtZ%2F5JnaZW2lt0c65%2F595jD2%2FeauHtmTZcyREIAwQ8JqkqudyMwPNy5g%2BAZyyRJRFKshE4MANuxkNDatHgSWu38om9tsxYzx0kDT7%2BGIFWS6yI4QZLUjODLcWLycM9jgKEiTFMW4cDl5bScMfaWttgCLuuC7peoPQGRgghiDLoVEfJF%2FAO0XzOaLSyiipxbXlzM32ACCHqHxFO4QjzS%2BNXLs8r%2BIxSnEUG%2F1gu5%2F78cbEE3uQ63a2Spq2ZXjC955Q9P92fDRjn4HdrbKf0zvg14TIwUnWVIDtGVd201h0ZuC9YsRIKteFuUfl0BJodL1fz%2FC5V9DVdoXRL9Ow76VWLqN2Jw7flr9l6jh4H67uXfbKKcwq8l2us0THW3JiW5fIYpnUlFPX9MPTDbBkmOM5wNAjCJFsDb%2BrC5JLYU%2Bc%2FjgMqVFs6t5oRSpkxJ6u0lD7dcHguQWMUPKYEy7RfUJQUlMSkqJIkSVMSJiQMWRL1YpaiChGaxhVi%2FSSitGQkS4u4oEVMSZFlPXC%2Ba

/var/folders/sb/n8yqrs6n7y9ghs7jf_2n7h280000gp/T/ipykernel_51280/2830233375.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(brand_query, conn)


         STATE    REGION         METRO     SPEND  IMPRESSIONS  CLICKS  \
48    New York  New York  New York, NY   5247.86         9006    2602   
163   New York  New York  New York, NY  15427.24         5246    2025   
216   New York  New York  New York, NY   8888.67         7372    2094   
887   New York  New York  New York, NY   6109.01         6947    1882   
1391  New York  New York  New York, NY   5338.73         5877    2005   

     MONTH_START  
48    2023-01-01  
163   2024-07-01  
216   2023-08-01  
887   2022-10-01  
1391  2023-05-01  
